<a href="https://colab.research.google.com/github/lavanya-shende/Predicting-Fake-news/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing


In [ ]:
#loading into pandas DataFrame
news_dataset = pd.read_csv('/content/test.csv')

In [ ]:
!kaggle competitions download -c fake-news

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
news_dataset = pd.read_csv('train.csv', engine='python', error_bad_lines=False)

<ipython-input-19-bd97bbf88e63>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  news_dataset = pd.read_csv('train.csv', engine='python', error_bad_lines=False)
Skipping line 13535: field larger than field limit (131072)
Skipping line 16544: unexpected end of data
Skipping line 13541: Expected 5 fields in line 13541, saw 6


In [ ]:
news_dataset.shape

(16540, 5)

In [ ]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [ ]:
#counting number of missing value
news_dataset.isnull().sum()

id           0
title      432
author    1592
text        49
label       16
dtype: int64

In [ ]:
#replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
#merging the author and title
news_dataset['content'] = news_dataset['author']+' '+ news_dataset['title']
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
16535     Ukrainian Foreign Ministry Protests Against P...
16536    b Reward Clinton's Hawkishness Because Trump's...
16537    Noah Remnick 11-Year-Old Grills Mike Pence on ...
16538    Anonymous Coward (UID 118243) World's Oldest P...
16539     Calais Jungle already turned into luxury apar...
Name: content, Length: 16540, dtype: object


In [ ]:
X = news_dataset.drop(columns='label',axis=1)
Y = news_dataset['label']

Stemming : reducing a word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
16535    ukrainian foreign ministri protest putin crime...
16536    b reward clinton hawkish trump foreign polici ...
16537    noah remnick year old grill mike penc soften d...
16538     anonym coward uid world oldest person smoke year
16539                calai jungl alreadi turn luxuri apart
Name: content, Length: 16540, dtype: object


In [ ]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(Y)

[1.0 0.0 1.0 ... 0.0 1.0 1.0]


In [ ]:
 #converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 14138)	0.2838355314271864
  (0, 12125)	0.2566423158250702
  (0, 8033)	0.36738824164061207
  (0, 7787)	0.2877419948429959
  (0, 6956)	0.24499640263474753
  (0, 6347)	0.21958807411118167
  (0, 4513)	0.23305607888006008
  (0, 3448)	0.27159449568927957
  (0, 3273)	0.36738824164061207
  (0, 2698)	0.24559996691800615
  (0, 2266)	0.3623765065105733
  (0, 250)	0.26994717118573797
  (1, 15129)	0.29953956584450114
  (1, 6177)	0.18929808473252369
  (1, 5003)	0.7179390189517463
  (1, 3244)	0.2643834331477672
  (1, 2563)	0.18949743897391472
  (1, 2031)	0.38145071467082586
  (1, 1731)	0.15356932596084974
  (1, 1376)	0.29001686843572455
  (2, 14073)	0.4165685151051023
  (2, 8658)	0.48975291299834156
  (2, 5416)	0.3474553299031292
  (2, 4897)	0.38323317427733655
  (2, 2828)	0.4661922202236992
  :	:
  (16537, 14050)	0.11823663403786523
  (16537, 13779)	0.09332077756634145
  (16537, 12708)	0.4035395051827526
  (16537, 11267)	0.3735509577635194
  (16537, 10096)	0.2878442591678688
  (16537, 9582)	0.

In [ ]:
Y = Y.astype(int)

Splitting the dataset into training and test data

In [ ]:
X_train, X_test, Y_train ,Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state= 2)

Trainig the Model using Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluating the model

In [ ]:
#accuracy for training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('accuracy score of training data : ', training_data_accuracy)

accuracy score of training data :  0.9913089480048367


In [ ]:
#accuracy for test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('accuracy score of test data : ', test_data_accuracy)

accuracy score of test data :  0.9815598548972189


Making a predictive system

In [ ]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('news is real')
else:
  print('news is fake')

[1]
news is fake
